## This notebook walks through the basics of generating synthetic choice data and fitting the model to that data to recover the paramaters that generated it

Fitting these models to data to choice data alone can be computational expensive. We fit the model below using 44 cpus.

`using` is the julia version of python's `import`, i.e. in incorporates the exported functions of a module into the current namespace. `Distributed` is a julia module for performing parallel computing. `addprocs()` adds some workers, which can be called by a main process if there are calls to do things in parallel. `PulseInputDDM` parallelizes the computation of the log-likelhood across trials.

In [1]:
using Distributed
addprocs(8);

### Fitting a choice model.

when `using` is called after workers have been made, i.e. above, then those modules are available on all the workers (i.e. any function called from those modules will be able to be executed on any worker)

In [2]:
using PulseInputDDM, Flatten

### Generate some data

Create an instance of the `θchoice` [composite type](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) which contains the 9 parameters of a choice DDM model. 

In [3]:
θ_syn = θchoice(θz=θz(σ2_i = 1., B = 13., λ = -0.5, σ2_a = 10., σ2_s = 1.0,
    ϕ = 0.4, τ_ϕ = 0.02), bias=0.1, lapse=0.1);

In [4]:
xgen = collect(Flatten.flatten(θ_syn));

Generate 20K trials of the synthetic data using those parameters. change `rng` to get a different set with the same parameters. `dt` specifies the temporal binning of the data. `1e-2` has worked well.

In [5]:
_, data = synthetic_data(;θ=θ_syn, ntrials=8_000, rng=2, dt=1e-2);

`PulseInputDDM` computes the log-likelihood by propogating mass of the latent distribution in time. This is done numerically by dividing the distribution up into little temporal and spatial bins. `n` is the number of spatials bins to use. `n=53` seems to work well.

In [ ]:
n = 53

Create an instance of the `choiceDDM` composite type which contains the parameters and the data of a choice DDM model. 

In [ ]:
model = choiceDDM(θ=θ_syn, data=data, n=n);

to see the docstring for `choiceDDM` type `?`

In [ ]:
@doc choiceDDM

Compute the loglikelihood of the model under the generative parameters

In [ ]:
loglikelihood(model)

random initial state

In [ ]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.2, 0.008], [0.,0.01]);

use `Flatten.reconstruct` to place the intial state into the `θchoice()` type, with each variable going into the right place

In [ ]:
θ = Flatten.reconstruct(θchoice(), x0)

In [ ]:
model = choiceDDM(θ=θ, data=data, n=n);

Compute the loglikelihood of the model under the initial parameters

In [ ]:
loglikelihood(model)

Define bounds for each parameter for the optimization. Some of these are strict (i.e. variance > 0) and some are not (B < 100 seems reasonable, but B could of course be larger then 100.)

In [ ]:
lb = [0., 2.,  -5., 0.,   0.,  0., 0.005, -5.0, 0.0]
ub = [30., 100., 5., 200., 10., 1.2,  1., 5.0, 1.0];

`fit` specifies which parameters should be learned. If they are not learned, they remain at their value from `x0`, the initial point of the optimization.

In [ ]:
fit = vcat(trues(9));

Create an instance of the `choiceoptions` composite type, using `lb`, `ub`, and `fit`. `choiceoptions` specifies which parameters to fit, and the lower and upper bounds of the optimization.

In [ ]:
options = choiceoptions(fit=fit, lb=lb, ub=ub)

In [ ]:
@doc choiceoptions

Define bounds for each parameter for the optimization. Some of these are strict (i.e. variance > 0) and some are not (B < 100 seems reasonable, but B could of course be larger then 100.)

### Optimize the model, using the options to define details of the optimization

In [ ]:
@doc PulseInputDDM.optimize

In [ ]:
model, output = optimize(model, options, f_tol=1e-3, iterations = 100, extended_trace=true, show_trace=true)

Compute the likelihood of the optimized model

In [20]:
loglikelihood(model)

-2964.391048661696

Compute the gradient of the model after optimization, to ensure we are near the minimum of the loglikelihood

In [21]:
gradient(model)

9-element Vector{Float64}:
 -0.035268930600005535
  0.08999858784521625
 -0.6998347283973927
  0.09059444267566338
  1.9425524414020203
  9.990898188423424
 -3.0944021983335404
 -1.2361640079379252
  5.96964709242075

Compute the Hessian of the model, to compute confidence bounds around the ML parameters

In [22]:
H = Hessian(model)
CI, HPSD = CIs(H);

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 0.523128861211172
└ @ PulseInputDDM /Users/briandepasquale/Documents/GitHub/PulseInputDDM/src/base_model.jl:19


Check the eigenvalues of the hessian, to ensure that it is positive semidefinite

In [23]:
using LinearAlgebra
eigvals(H)

9-element Vector{Float64}:
     -0.050025419469309766
     -0.0003796327464849516
      0.00412042590508605
      4.732316790901145
     14.677064922676797
     37.879545011449025
    115.10801793426742
  15264.399969525104
 402944.66090952803

In [24]:
xf = collect(Flatten.flatten(model.θ));

Check that the solution is within the confidence intervals

In [25]:
(xf - CI) .< xgen, (xf + CI) .> xgen

(Bool[1, 1, 1, 1, 1, 0, 1, 1, 1], Bool[1, 1, 1, 1, 1, 1, 1, 1, 0])

In [31]:
output

 * Status: success

 * Candidate solution
    Final objective value:     2.974295e+03

 * Found with
    Algorithm:     Fminbox with BFGS

 * Convergence measures
    |x - x'|               = 1.94e-03 ≰ 1.0e-10
    |x - x'|/|x'|          = 6.41e-05 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 1.0e-03
    |g(x)|                 = 1.48e+00 ≰ 1.0e-03

 * Work counters
    Seconds run:   3584  (vs limit 170000)
    Iterations:    2
    f(x) calls:    26
    ∇f(x) calls:   16


Get the step-by-step values of the optimization and plot them

In [32]:
trace = hcat(map(x-> x.metadata["x"], output.trace)...);

In [33]:
using PyPlot

In [ ]:
num_rows, num_cols = 3,3
fig, ax = subplots(num_rows, num_cols, figsize=(9,9))
name = ["σ2_i", "B", "λ", "σ2_a", "σ2_s", "ϕ", "τ_ϕ", "bias", "lapse"]

for i in 1:9
                  
    ax[i].plot(trace[i,:])
    ax[i].plot(xgen[i] * ones(size(trace,2)))
    ax[i].set_title(name[i])
    
    ax[i].errorbar(size(trace, 2), xf[i], yerr=CI[i], fmt="o",
        capsize=6)
end

tight_layout() 
IJulia.display()

In [35]:
lb = options.lb
ub = options.ub;

Compute the LL over the optimization domain, for plotting

In [36]:
@everywhere ℓℓ(x, model) = -PulseInputDDM.loglikelihood(x, model)

In [37]:
αs = hcat(map((lb,ub)-> range(lb + eps(), stop=ub, length=30), lb, ub));

In [38]:
LL_scan = map(i-> map(x-> ℓℓ(vcat(xf[1:i-1], x, xf[i+1:end]), model), αs[i]), 1:9);

In [30]:
num_rows, num_cols = 3,3
fig, ax = subplots(num_rows, num_cols, figsize=(9,9))
name = ["σ2_i", "B", "λ", "σ2_a", "σ2_s", "ϕ", "τ_ϕ", "bias", "lapse"]

for i in 1:9
                  
    ax[i].plot(αs[i], LL_scan[i], "x")
    ax[i].set_title(name[i])
    ax[i].plot(xgen[i]*ones(100), range(minimum(LL_scan[i]), stop=maximum(LL_scan[i]), length=100), "k")
    ax[i].plot(xf[i]*ones(100), range(minimum(LL_scan[i]), stop=maximum(LL_scan[i]), length=100),
        "g--")
    ax[i].plot(max((xf[i] - CI[i]), lb[i]) *ones(100), range(minimum(LL_scan[i]), 
            stop=maximum(LL_scan[i]), length=100), "r--")
    ax[i].plot(min((xf[i] + CI[i]), ub[i]) *ones(100), range(minimum(LL_scan[i]), 
            stop=maximum(LL_scan[i]), length=100), "r--")

    
end

tight_layout() 

UndefVarError: UndefVarError: `αs` not defined

## Saving and reloading modeling results and saving data

Use the `save_choice_model` function to save the optimized model, the options used to define the optimization, and the confidence intervals

In [40]:
save_file = "../choice model/example_results.mat"
save_choice_model(save_file, model, options, CI)

if you want to restart the optimization from where you stopped `reload_choice_model` will reload those model parameters and the `options`

In [41]:
θ, options = reload_choice_model(save_file);

Save the choice data using `save_choice_data` and then reload it using `load_choice_data`

In [42]:
import PulseInputDDM: save_choice_data

In [43]:
save_choice_data("../choice model/example_matfile_2.mat", data)

In [44]:
data = load_choice_data("../choice model/example_matfile_2.mat");

In [45]:
model = choiceDDM(θ=θ, data=data, n=n);
loglikelihood(model)

-7435.988771905236

## Loading data and fitting a choice model from real data

Because many neuroscientists use matlab, we use the [MAT.jl](https://github.com/JuliaIO/MAT.jl) package for IO. Data can be loaded using two conventions. One of these conventions is easier when data is saved within matlab as a .MAT file, and is described below. 

The package expects your data to live in a single .mat file which should contain a struct called `rawdata`. Each element of `rawdata` should have data for one behavioral trial and `rawdata` should contain the following fields with the specified structure:

 - `rawdata.leftbups`: row-vector containing the relative timing, in seconds, of left clicks on an individual trial. 0 seconds is the start of the click stimulus.
 - `rawdata.rightbups`: row-vector containing the relative timing in seconds (origin at 0 sec) of right clicks on an individual trial. 0 seconds is the start of the click stimulus.
 - `rawdata.T`: the duration of the trial, in seconds. The beginning of a trial is defined as the start of the click stimulus. The end of a trial is defined based on the behavioral event “cpoke_end”. This was the Hanks convention.
 - `rawdata.pokedR`: `Bool` representing the animal choice (1 = right).
 
The example file located at `../choice model/example_matfile.mat` adheres to this convention and can be loaded using the `load_choice_data` method.

In [46]:
data = load_choice_data("../choice model/example_matfile.mat");

In [47]:
θ2 = θchoice(θz=θz(σ2_i = 2., B = 23., λ = 0.5, σ2_a = 100., σ2_s = 1.1,
    ϕ = 0.35, τ_ϕ = 0.035), bias=0.2, lapse=0.01);

In [48]:
model = choiceDDM(θ=θ2, data=data, n=n);
loglikelihood(model)

-4.321480367653435